In [ ]:
#training model for image directories
import os #operating system functions
import cv2 #computer vision
import numpy as np #numerical python for creating arrays
from PIL import Image #Pillow for image manipulation
import pickle #serializing and de-serializing 

BASE_DIR = cwd = os.getcwd() #os.getcwd() gets the location of the cwd
img_dir = os.path.join(BASE_DIR, "images") #joins 2 paths

face_cascade = cv2.CascadeClassifier('C:/Users/Rujuta/ML OpenCv project/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()


current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root,dirs,files in os.walk(img_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
#             print(label, path)
            if label in label_ids:
                pass
            else:
                label_ids[label] = current_id
                current_id += 1
                
            id_ = label_ids[label]
#             print(label_ids)
#             y_labels.append(lable) #some number
#             x_train.append(path) #verify this image, turn into NUMPY array,GRAY
            pil_image = Image.open(path).convert("L") #grayscale
            size = (250,250)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
    
            
            image_array = np.array(final_image, "uint8")
#             print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y: y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
                
                
# print(y_labels)
# print(x_train)

with open("labels.pickle", 'wb') as f:
    pickle.dump(label_ids, f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")
    

In [1]:
#Face detection
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('C:/Users/Rujuta/ML OpenCv project/data/haarcascade_frontalface_alt2.xml') #detect objects in a video stream
recognizer = cv2.face.LBPHFaceRecognizer_create() #Recognise
recognizer.read("trainer.yml") #read trained model file

labels = {"person_name": 1} #Labels dictionary initiated
with open("labels.pickle", 'rb') as f:  #read stored labels
    og_labels =   pickle.load(f) #load labels
    labels = {v:k for k,v in og_labels.items()}


cap = cv2.VideoCapture(0) #Open a camera for video capturing

while(True):
    
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    
    for(x,y,w,h) in faces:
#         print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w] #(ycordinate_start,ycordinate_end)
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf>= 45 and conf<= 85:
#             print(id_)
#             print(labels[id_])
      
            #shape,size,colour of rectangular box
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255,255,255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)
        
       
        img_item = "6.png"
        cv2.imwrite(img_item, roi_gray)
        
        #Rectangle around frame
        color = (255,0,0) #BGR not RGB (Blue color)
        stroke = 2
        end_cord_x = x+w
        end_cord_y = y+h
        cv2.rectangle(frame, (x,y), (end_cord_x,end_cord_y), color, stroke)
    
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
        
name = labels[id_]
print(name)  

cap.release()
cv2.destroyAllWindows()

rujuta-vaidya


In [ ]:
#QRCODE RECOGNITION
import cv2
import numpy as np
from pyzbar.pyzbar import decode


cap =cv2.VideoCapture(0)
cap.set(3, 640)  #sets width of frame
cap.set(4, 480)  #sets height of frame

while True:

    success , img =cap.read()

    for barcode in decode(img):
        mydata = barcode.data.decode('utf-8')
#         print(mydata)
        pts =  np.array([barcode.polygon], np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(img, [pts] ,True, (225 ,0, 0), 4)

        
    cv2.imshow("frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print(mydata)
cap.release()
cv2.destroyAllWindows()

In [ ]:
from tkinter import *
from PIL import ImageTk,Image   
if(name==mydata):
    msg="Successfully Validated!"
    msgColor = "#3efa2d"
else:
    msg = "Invalid. Try Again :("
    msgColor = "#000000"
window =Tk()
window.geometry("500x200")
window.title("Validator")

#displaying message 
message = Message(window, text= msg)
message.config(font=("Consolas", 15, "bold"))
message.config(aspect =550)
message.config(fg=msgColor) #foreground color 
#message.config(relief =SUNKEN) #frame styling
message.config(padx = 50)     #padding on x axis
message.config(pady = 70)     #padding on y axis

message.pack()
#message.place(x=80,y=10)
window.mainloop()